In [1]:
import torch
from cnn_model.Vgg import VGG 
from cnn_model.Alexnet import AlexNet


In [2]:
torch.cuda.is_available

<function torch.cuda.is_available() -> bool>

In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alex_model = AlexNet().to(DEVICE) # input img size = (227,227)
list(alex_model.children())

[Sequential(
   (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
   (1): ReLU()
   (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
 ),
 Sequential(
   (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
   (1): ReLU()
   (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
 ),
 Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 Sequential(
   (0): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (1): ReLU()
   (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
 ),
 Linear(in_features=9216, out_features=4096, bias=True),
 Linear(in_features=4096, out_features=4096, bias=True),
 Linear(in_features=4096, out_features=10, bias=True)]

In [4]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg16_model = VGG(3,10,'vgg16').to(DEVICE)
list(vgg16_model.children())

[Sequential(
   (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (1): ReLU()
   (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (3): ReLU()
   (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 ),
 Sequential(
   (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (1): ReLU()
   (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (3): ReLU()
   (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 ),
 Sequential(
   (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (1): ReLU()
   (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (3): ReLU()
   (4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (5): ReLU()
   (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 ),
 Sequential(
   (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 

In [5]:
vgg16_model._modules.keys()

odict_keys(['conv1', 'conv2', 'conv3', 'conv4', 'conv5', 'fc_layer'])

In [6]:
import math 
import os 
import numpy as np 
import matplotlib.pyplot as plt
import datetime

import torch
import torch.nn as nn 
import torchvision
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader 
import torchvision.transforms as transforms 
from torchvision import datasets

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.491,0.482, 0.447), (0.247, 0.243, 0.261)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train = True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True,num_workers = 2)

testset = torchvision.datasets.CIFAR10(root='./data', train = False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False,num_workers = 2)



KeyboardInterrupt: 

[0.49139968 0.48215841 0.44653091]
[0.24703223 0.24348513 0.26158784]

In [ ]:
print("-----Train_datasets----- \n",trainset)
print("-----Test_datasets----- \n",testset)

for X, y in trainloader:
    print('-'*30)
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

-----Train_datasets----- 
 Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               Resize(size=224, interpolation=bilinear)
               ToTensor()
               Normalize(mean=(0.491, 0.482, 0.447), std=(0.247, 0.243, 0.261))
           )
-----Test_datasets----- 
 Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               Resize(size=224, interpolation=bilinear)
               ToTensor()
               Normalize(mean=(0.491, 0.482, 0.447), std=(0.247, 0.243, 0.261))
           )
------------------------------
Shape of X [N, C, H, W]: torch.Size([64, 3, 224, 224])
Shape of y: torch.Size([64]) torch.int64


In [ ]:
# import torch.optim as optim
criterion = nn.CrossEntropyLoss().to(DEVICE)
optimizer = optim.Adam(vgg16_model.parameters(),lr = 0.0001)

In [ ]:
EPOCH = 30

In [ ]:
loss_arr = [] 
for epoch in range(EPOCH):
    runnig_loss = 0.0 

    # if (epoch>0):
    #     model = VGG16(num_classes = NUM_CLASS).to(DEVICE)
    #     model.load_state_dict(torch.load(save_path))
    #     model.to(DEVICE)

    for i,data in enumerate(trainloader,0):
        input,labels = data 
        input,labels = input.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs= vgg16_model(input)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step() 

        if(loss.item()>1000):
            print(loss.item())
            for param in vgg16_model.parametes():
                print(param.data)
                break

        runnig_loss +=loss.item()
        if i % 50 == 49:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, runnig_loss / 50))
            loss_arr.append(runnig_loss)
            runnig_loss = 0.0 
    
    #save_path=SAVE_PATH + "cifar_vgg16.pth"
    #torch.save(vgg16_model.state_dict(), save_path)

print('Finished Training')



[1,    50] loss: 2.303
[1,   100] loss: 2.303
[1,   150] loss: 2.303
[1,   200] loss: 2.304
[1,   250] loss: 2.250
[1,   300] loss: 2.072
[1,   350] loss: 1.979
[1,   400] loss: 1.898
[1,   450] loss: 1.858
[1,   500] loss: 1.808
[1,   550] loss: 1.773
[1,   600] loss: 1.728
[1,   650] loss: 1.720
[1,   700] loss: 1.668
[1,   750] loss: 1.661
[2,    50] loss: 1.600
[2,   100] loss: 1.528
[2,   150] loss: 1.550
[2,   200] loss: 1.511
[2,   250] loss: 1.495
[2,   300] loss: 1.481
[2,   350] loss: 1.434
[2,   400] loss: 1.422
[2,   450] loss: 1.393
[2,   500] loss: 1.395
[2,   550] loss: 1.395
[2,   600] loss: 1.325
[2,   650] loss: 1.301
[2,   700] loss: 1.303
[2,   750] loss: 1.270
[3,    50] loss: 1.208
[3,   100] loss: 1.207
[3,   150] loss: 1.190
[3,   200] loss: 1.149
[3,   250] loss: 1.177
[3,   300] loss: 1.130
[3,   350] loss: 1.131
[3,   400] loss: 1.127
[3,   450] loss: 1.087
[3,   500] loss: 1.093
[3,   550] loss: 1.054
[3,   600] loss: 1.073
[3,   650] loss: 1.033
[3,   700] 

In [ ]:
LABEL_NAME = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
NUM_CLASS = len(LABEL_NAME)

In [ ]:
class_correct = list(0. for i in range(NUM_CLASS))
class_total = list(0. for i in range(NUM_CLASS))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        outputs= vgg16_model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(NUM_CLASS):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

accuracy_sum=0
for i in range(10):
    temp = 100 * class_correct[i] / class_total[i]
    print('Accuracy of %5s : %2d %%' % (LABEL_NAME[i], temp))

    accuracy_sum+=temp
print('Accuracy average: ', accuracy_sum/NUM_CLASS)

Accuracy of airplane : 77 %
Accuracy of automobile : 88 %
Accuracy of  bird : 51 %
Accuracy of   cat : 58 %
Accuracy of  deer : 72 %
Accuracy of   dog : 74 %
Accuracy of  frog : 76 %
Accuracy of horse : 80 %
Accuracy of  ship : 91 %
Accuracy of truck : 83 %
Accuracy average:  75.30876799590796


In [ ]:
torch.save(vgg16_model,'./vgg_pt//vgg16_cifar10_30epoch.pt')